## Импорт библиотек

In [15]:
import pandas as pd
import os
import warnings
import numpy as np
import re

from functools import reduce
from datetime import datetime
from datetime import date

warnings.filterwarnings("ignore")

## Определение функций

## Установка параметров

In [16]:
marks_for_parsing = ['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
                     'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI']
data_columns = ['bodyType', 'brand', 'car_url', 'color', 'engineDisplacement', 'enginePower', 
                'equipment_dict','fuelType', 'mileage', 'modelDate', 'model_name', 'numberOfDoors', 
                'productionDate', 'sell_id', 'vehicleTransmission', 'vendor', 
                'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль']
outer_datasets = ['all_auto_ru_09_09_2020.csv',
                     'all_car_27_08_2021_data_12_brands.csv',
                     'data1.csv',
                     'parsing_auto_ru.csv',
                     'train_df_full_part1.csv']

### Сокращение (перезапись только брендов из test)

In [17]:
# for df_name,pars_date in  zip(outer_datasets,parsing_date):
#     try:
#         df = pd.read_csv('./Preproject_7_data/' + df_name,low_memory=False)
#     except Exception:
#         df = pd.read_csv('./Preproject_7_data/' + df_name,sep='\t',low_memory=False) 
#     df = df[df.brand.apply(lambda x:x in marks_for_parsing)]
#     file_name = 'auto_ru_' + pars_date + '.csv'
#     print(f'./Project_7_data/{file_name}')
#     df.to_csv(f'./Project_7_data/{file_name}',index=False)

In [18]:
datasets_to_train = []
for df_name,pars_date in  zip(outer_datasets,parsing_date):
    file_name = 'auto_ru_' + pars_date + '.csv'
    datasets_to_train.append(file_name)

### Определение даты каждого датсета/ курса на эту дату

In [19]:
parsing_date = ['2020_09_09','2021_08_29','2020_10_19','2021_08_13','2021_10_01']
currency_rate = [1,1,1,1,1]
basket_currency_rate_ = [1,1,1,1,1]

### Корреспонденция колонок

### auto_ru_2020_09_09.csv

In [20]:
test = pd.read_csv('./Project_7_data/test.csv',low_memory=False)

In [21]:
df= pd.read_csv('./Project_7_data/' + datasets_to_train[0],low_memory=False)

In [22]:
test.columns

Index(['bodyType', 'brand', 'car_url', 'color', 'complectation_dict',
       'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
       'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name',
       'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency',
       'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС',
       'Привод', 'Руль', 'Состояние', 'Таможня'],
      dtype='object')

In [23]:
df.columns

Index(['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name',
       'numberOfDoors', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'engineDisplacement', 'enginePower',
       'description', 'mileage', 'Комплектация', 'Привод', 'Руль', 'Состояние',
       'Владельцы', 'ПТС', 'Таможня', 'Владение', 'price', 'start_date',
       'hidden', 'model'],
      dtype='object')

## ============================================

#### bodyType

In [41]:
def fghj(x):
    res=[]
    try:
        for body_type in test.bodyType.unique():
            if body_type in x:
                res.append(body_type)
    except Exception: return x
    return max(res)

In [43]:
%%time
lambda x: x.lower() if x else x
# df.bodyType.apply(lambda x: x.lower() if type(x) == str else x)[:15]
ttt = df.bodyType.apply(lambda x: x.lower() if type(x) == str else x).apply(fghj)
ttt.unique()

Wall time: 1min 28s


array(['седан', 'универсал 5 дв.', 'хэтчбек 5 дв.', 'хэтчбек 3 дв.',
       'кабриолет', 'купе', 'лифтбек', 'внедорожник 5 дв.', 'седан 2 дв.',
       'компактвэн', 'купе-хардтоп', 'родстер', 'микровэн', nan,
       'минивэн', 'внедорожник 3 дв.', 'пикап двойная кабина', 'тарга',
       'фастбек', 'пикап одинарная кабина', 'пикап полуторная кабина',
       'седан-хардтоп', 'фургон', 'внедорожник открытый', 'лимузин'],
      dtype=object)

In [131]:
aaa = 'седан 2 дв.'
res = []
for t in ttt:
    if t in aaa:
        res.append(t)
max(res)
# 'внедорожник 5 дв. long'

'седан 2 дв.'

In [119]:
ttt = test.bodyType.unique()
ttt

array(['лифтбек', 'внедорожник 5 дв.', 'хэтчбек 5 дв.', 'седан',
       'компактвэн', 'универсал 5 дв.', 'пикап одинарная кабина',
       'хэтчбек 3 дв.', 'купе', 'кабриолет', 'минивэн',
       'пикап двойная кабина', 'внедорожник 3 дв.', 'родстер', 'микровэн',
       'седан 2 дв.', 'купе-хардтоп', 'фастбек', 'тарга',
       'внедорожник открытый', 'лимузин', 'пикап полуторная кабина',
       'седан-хардтоп', 'фургон'], dtype=object)

### Стандартизация датасетов

In [116]:
sorted(test.bodyType.unique())

['внедорожник 3 дв.',
 'внедорожник 5 дв.',
 'внедорожник открытый',
 'кабриолет',
 'компактвэн',
 'купе',
 'купе-хардтоп',
 'лимузин',
 'лифтбек',
 'микровэн',
 'минивэн',
 'пикап двойная кабина',
 'пикап одинарная кабина',
 'пикап полуторная кабина',
 'родстер',
 'седан',
 'седан 2 дв.',
 'седан-хардтоп',
 'тарга',
 'универсал 5 дв.',
 'фастбек',
 'фургон',
 'хэтчбек 3 дв.',
 'хэтчбек 5 дв.']

In [118]:
test.bodyType.value_counts()

седан                      13147
внедорожник 5 дв.          12228
лифтбек                     2232
хэтчбек 5 дв.               2124
универсал 5 дв.             1276
минивэн                      995
купе                         824
компактвэн                   489
хэтчбек 3 дв.                485
пикап двойная кабина         280
купе-хардтоп                 158
внедорожник 3 дв.            115
родстер                       99
фургон                        97
кабриолет                     90
седан-хардтоп                 12
микровэн                       7
лимузин                        6
седан 2 дв.                    6
пикап полуторная кабина        5
пикап одинарная кабина         5
внедорожник открытый           3
тарга                          2
фастбек                        1
Name: bodyType, dtype: int64

### Анализ распределения цены в 5(10) топ моделях

In [22]:
for df_name in  datasets_to_train:
    try:
        df = pd.read_csv('./Preproject_7_data/' + df_name,low_memory=False)
    except Exception:
        df = pd.read_csv('./Preproject_7_data/' + df_name,sep='\t',low_memory=False) 
    df = df[df.brand.apply(lambda x:x in marks_for_parsing)]
    
    print(df.columns)
#     models_for_distribution = df.model_name.value_counts().index[:5].to_list()
#     print(models_for_distribution)

Index(['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name',
       'numberOfDoors', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'engineDisplacement', 'enginePower',
       'description', 'mileage', 'Комплектация', 'Привод', 'Руль', 'Состояние',
       'Владельцы', 'ПТС', 'Таможня', 'Владение', 'price', 'start_date',
       'hidden', 'model'],
      dtype='object')
Index(['Unnamed: 0', 'car_url', 'bodyType', 'brand', 'color',
       'complectation_dict', 'description', 'engineDisplacement',
       'enginePower', 'equipment_dict', 'fuelType', 'image', 'mileage',
       'modelDate', 'model_name', 'name', 'numberOfDoors', 'parsing_unixtime',
       'price', 'priceCurrency', 'productionDate', 'sell_id', 'views',
       'date_added', 'super_gen', 'vehicleConfiguration',
       'vehicleTransmission', 'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль',
       'Состояние', 'Таможня', 'region'],
      dtype='object')
Index(['bodyType', 'brand', 'car_url', 'col

In [20]:
models_for_distribution = df.model_name.value_counts().index[:5].to_list()
models_for_distribution

['Tiguan', 'X-Trail', 'Land Cruiser', 'Touareg', 'Polo']

### Соотношение курсового коэфициента с рапределением

In [10]:
'''for df_name in  datasets_to_train:
    
    try:
        df = pd.read_csv('./Preproject_7_data/' + df_name,low_memory=False)
#         print('ok')
    except:
        df = pd.read_csv('./Preproject_7_data/' + df_name,sep='\t',low_memory=False) 
#         print('trouble')
#     print(df.brand.unique())
    df = df[df.brand.apply(lambda x:x in marks_for_parsing)]
#     print(df.brand.unique())
    print(df_name,end=' ')
    try:
        print(date.fromtimestamp((df.parsing_unixtime.min())))
    except:
        print('no parsing_unixtime')
    print(df_name,end=' ')
    try:
        print(date.fromtimestamp((df.parsing_unixtime.max())))
    except:
        print('no parsing_unixtime')''';

## ==================================

In [27]:
def trfn(x):
    try:res = x.lower()   
    except Exception:res = x
    return res

df.bodyType.apply(trfn).unique()[:15]

array(['седан', 'универсал 5 дв.', 'хэтчбек 5 дв. sportback',
       'хэтчбек 3 дв.', 'хэтчбек 5 дв.', 'кабриолет', 'купе',
       'лифтбек sportback', 'лифтбек', 'седан long', 'внедорожник 5 дв.',
       'кабриолет roadster', 'седан 2 дв.', 'седан gran coupe',
       'компактвэн'], dtype=object)